In [ ]:
from utils.utils import JIGSAW_DATASET_DIR_RAW, JIGSAW_DATASET_DIR_PROCESSED
path_to_save_raw_dataset = JIGSAW_DATASET_DIR_RAW
path_to_save_processed_dataset = JIGSAW_DATASET_DIR_PROCESSED
%pip install kaggle
# create a kaggle.json file with your API token at ~/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification   \
    --path {path_to_save_raw_dataset}

!unzip {path_to_save_raw_dataset}/jigsaw-unintended-bias-in-toxicity-classification.zip
!rm -rf {path_to_save_raw_dataset}/jigsaw-unintended-bias-in-toxicity-classification.zip

In [ ]:
from datasets import load_dataset
dataset_name = "google/jigsaw_unintended_bias"
# generate dataset from csv files
dataset = load_dataset(dataset_name, data_dir=path_to_save_raw_dataset)

In [ ]:
# rename test_public_leaderboard to val and test_private_leaderboard to test
dataset["val"] = dataset.pop("test_public_leaderboard")
dataset["test"] = dataset.pop("test_private_leaderboard")



In [26]:
# extract black/white, female/male, christian/muslim/jewish datasets
train_dataset_black = dataset["train"].filter(lambda x: x["black"] > 0.5)
val_dataset_black = dataset["val"].filter(lambda x: x["black"] > 0.5)
test_dataset_black = dataset["test"].filter(lambda x: x["black"] > 0.5)

train_dataset_white = dataset["train"].filter(lambda x: x["white"] > 0.5)
val_dataset_white = dataset["val"].filter(lambda x: x["white"] > 0.5)
test_dataset_white = dataset["test"].filter(lambda x: x["white"] > 0.5)

train_dataset_male = dataset["train"].filter(lambda x: x["male"] > 0.5)
val_dataset_male = dataset["val"].filter(lambda x: x["male"] > 0.5)
test_dataset_male = dataset["test"].filter(lambda x: x["male"] > 0.5)

train_dataset_female = dataset["train"].filter(lambda x: x["female"] > 0.5)
val_dataset_female = dataset["val"].filter(lambda x: x["female"] > 0.5)
test_dataset_female = dataset["test"].filter(lambda x: x["female"] > 0.5)

train_dataset_christian = dataset["train"].filter(lambda x: x["christian"] > 0.5)
val_dataset_christian = dataset["val"].filter(lambda x: x["christian"] > 0.5)
test_dataset_christian = dataset["test"].filter(lambda x: x["christian"] > 0.5)

train_dataset_muslim = dataset["train"].filter(lambda x: x["muslim"] > 0.5)
val_dataset_muslim = dataset["val"].filter(lambda x: x["muslim"] > 0.5)
test_dataset_muslim = dataset["test"].filter(lambda x: x["muslim"] > 0.5)

train_dataset_jewish = dataset["train"].filter(lambda x: x["jewish"] > 0.5)
val_dataset_jewish = dataset["val"].filter(lambda x: x["jewish"] > 0.5)
test_dataset_jewish = dataset["test"].filter(lambda x: x["jewish"] > 0.5)

Filter:   0%|          | 0/1804874 [00:00<?, ? examples/s]

Filter: 100%|██████████| 97320/97320 [00:02<00:00, 32610.12 examples/s]


In [27]:
# create a new dataset, the dataset consists of subsets of the above groups, and each subset consists of train, val, and test split

new_dataset = {"black": {"train": train_dataset_black, "val": val_dataset_black, "test": test_dataset_black},
              "white": {"train": train_dataset_white, "val": val_dataset_white, "test": test_dataset_white},
              "female": {"train": train_dataset_female, "val": val_dataset_female, "test": test_dataset_female},
              "male": {"train": train_dataset_male, "val": val_dataset_male, "test": test_dataset_male},
              "christian": {"train": train_dataset_christian, "val": val_dataset_christian, "test": test_dataset_christian},
              "muslim": {"train": train_dataset_muslim, "val": val_dataset_muslim, "test": test_dataset_muslim},
              "jewish": {"train": train_dataset_jewish, "val": val_dataset_jewish, "test": test_dataset_jewish}
                }

# convert to a huggingface dataset
from datasets import DatasetDict
new_dataset = DatasetDict(new_dataset)

In [29]:
# keep only one label: if target is greater than 0.5, then the label is 1, otherwise 0
def convert_to_binary(example):
    example["label"] = 1 if example["target"] > 0.5 else 0
    return example

for subset in new_dataset:
    for split in new_dataset[subset]:
        new_dataset[subset][split] = new_dataset[subset][split].map(convert_to_binary)

Map:   0%|          | 0/13869 [00:00<?, ? examples/s]

Map: 100%|██████████| 387/387 [00:00<00:00, 1868.28 examples/s]


In [30]:
# rename comment_text to text, keep only text and label columns
for subset in new_dataset:
    for split in new_dataset[subset]:
        new_dataset[subset][split] = new_dataset[subset][split].rename_column("comment_text", "text")
        new_dataset[subset][split] = new_dataset[subset][split].select_columns(["text", "label"])



In [ ]:
# save the new dataset
import os
for subset_name, split_dict in new_dataset.items():
    dataset_dict = DatasetDict(split_dict)
    dataset_dict.save_to_disk(os.path.join(path_to_save_processed_dataset, subset_name))

Saving the dataset (0/1 shards):   0%|          | 0/13869 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 387/387 [00:00<00:00, 15987.04 examples/s]
